In [1]:
import pandas as pd
import os
from datetime import datetime
import time
from os.path import exists
from time import sleep
import requests
from tqdm import tqdm
import requests
import random
import json
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Spotify-Data-Preprocessing

### Functions

In [32]:
def load_append_jsons(number):
    """
    Combines multiple json files into one dataframe and save the dataframe as a new csv-file.
    :param int number: number of jsons
    :return: dataframe with combined json files
    """

    # initialize first dataframe
    df_final = pd.read_json(r'endsong_0.json')

    # loop to append the other jsons to one dataframe
    for i in range(1, number):
        df_add = pd.read_json(f'endsong_{i}.json')
        df_final = df_final.append(df_add, ignore_index = True)
        #print("working")

    # save as a new file
    filename = "spotify_data.xlsx"

    if os.path.exists(filename):
        print("File already exists!")
        pass
    else:
        df_final.to_excel(filename, encoding="utf-8")
        print("File was saved!")

    return df_final


In [ ]:
def cleaning(df):
    """
    Cleaning of dataframe to get additional numeric values in different time formats.
    :param df: dataframe as input
    :return: cleaned dataframe
    """
    # convert to datetime datatype
    df['ts']= pd.to_datetime(df['ts'])

    # extend dataframe
    df['year'] = pd.DatetimeIndex(df['ts']).year
    df['second_played'] = df['ms_played'] / 1000
    df['minutes_played'] = df['second_played'] / 60
    df['uri'] = df["spotify_track_uri"].str.split(':').str[2]
    df['pod_uri'] = df["spotify_episode_uri"].str.split(':').str[2]

    # rearange dataframe
    df_final = df[['ts', 'year', 'username', 'platform','master_metadata_album_artist_name', 'master_metadata_album_album_name', 'master_metadata_track_name','uri', 'spotify_track_uri', 'ms_played', 'second_played', 'minutes_played', 'conn_country', 'ip_addr_decrypted', 'user_agent_decrypted', 'episode_name', 'episode_show_name', 'spotify_episode_uri', 'pod_uri', 'reason_start', 'reason_end', 'shuffle', 'skipped','offline', 'offline_timestamp', 'incognito_mode']]

    # delete null values
    indexTrack = df_final[(df_final['master_metadata_album_artist_name'].isna()) & (df_final['episode_show_name'].isna())].index
    df_final.drop(indexTrack , inplace=True)

    return df_final

In [ ]:
def splitting(df):
    """
    Split the dataframe into music streaming history and podcast streaming history
    :param df: dataframe as input
    :return: two dataframes for music and podcasts
    """
    # splitting dataframe into podcasts and tracks
    # dataframe for podcasts
    df_podcasts = df[df['master_metadata_album_artist_name'].isna()]
    # drop not relevant columns
    df_podcasts.drop(['master_metadata_album_artist_name', 'master_metadata_album_album_name', 'master_metadata_track_name', 'uri', 'spotify_track_uri'], axis=1, inplace=True)

    # dataframe for tracks
    df_tracks = df[df['episode_show_name'].isna()]
    # drop not relevant columns
    df_tracks.drop(['episode_name', 'episode_show_name', 'spotify_episode_uri', 'pod_uri'], axis=1, inplace=True)

    return df_tracks, df_podcasts

In [33]:
df_stream = load_append_jsons(9)

File was saved!


In [42]:
df_stream.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147512 entries, 0 to 147511
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ts                                 147512 non-null  object 
 1   username                           147512 non-null  int64  
 2   platform                           147512 non-null  object 
 3   ms_played                          147512 non-null  int64  
 4   conn_country                       147512 non-null  object 
 5   ip_addr_decrypted                  147512 non-null  object 
 6   user_agent_decrypted               147100 non-null  object 
 7   master_metadata_track_name         145372 non-null  object 
 8   master_metadata_album_artist_name  145372 non-null  object 
 9   master_metadata_album_album_name   145372 non-null  object 
 10  spotify_track_uri                  145372 non-null  object 
 11  episode_name                       1798

In [1]:
df_stream_clean = cleaning(df_stream)

In [2]:
df_stream_clean.head()

In [48]:
df_track, df_pod = splitting(df_stream_clean)

In [ ]:
#df_track.to_csv("music_streaming_data.csv", encoding="utf-8")
#df_pod.to_csv("podcast_streaming_data.csv", encoding="utf-8")

### Working with cleaned datasets

In [2]:
df_podcast = pd.read_csv("podcast_streaming_data.csv", index_col=[0])
df_podcast = df_podcast.reset_index(drop=True)
df_music = pd.read_csv("music_streaming_data.csv", index_col=[0])
df_music = df_music.reset_index(drop=True)

In [3]:
df_podcast.head()

In [4]:
df_music.head()

### Get Additional Infos with Spotify-API

#### API-Dokumentation-Link
https://developer.spotify.com/console/

**Creating-Developer-Account:**
https://medium.com/@maxtingle/getting-started-with-spotifys-api-spotipy-197c3dc6353b

In [2]:
# API-Setup
# generate access token
# save IDs from the new project in Spotify Developer Dashboard
CLIENT_ID = 'your-client-id'
CLIENT_SECRET = 'your-client-secret'

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# Post
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

### Preparation Steps
1. Find one song of each artist to get the track-uri, to be able to do api-calls to the track-endpoint
2. Extract artists-uri of the api-calls
3. Make api-calls to artists-endpoint to get additional features of artists like: genre, popularity, followers
4. Additional Cleaning-Steps

In [45]:
# get one track id for each artist
artists = df_music.master_metadata_album_artist_name.unique().tolist()
artists_track_uri = []

In [48]:
# search for one track-uri of each artist
for v in tqdm(artists):
    indices = df_music.index[df_music['master_metadata_album_artist_name'] == v]
    first_index = indices[0]
    if first_index:
        track_uri = df_music.iloc[first_index]['uri']
        #track_uri = df_music.at[df_music.index[first_index], 'uri']
        artists_track_uri.append(track_uri)
    else:
        artists_track_uri.append(None)

100%|██████████| 8371/8371 [00:56<00:00, 146.96it/s]


In [75]:
# save in file
df_artist_track_uri = pd.DataFrame(list(zip(artists, artists_track_uri)), columns =['artist', 'track_uri'])

In [7]:
df_artist_track_uri.head(20)

In [ ]:
### make get calls with track-endpoint to get id of each artist

In [56]:
# create list with track-uris
uri_list = df_artist_track_uri.track_uri.tolist()
# list to save infos of API-Calls of each requested track informations
track_info_list = []

In [65]:
# loop for each api-call of the tracks-endpoint
for t in tqdm(uri_list):
    track_info = requests.get(BASE_URL + 'tracks/' + t, headers=headers)
    track_info_json = track_info.json()
    track_info_list.append(track_info_json)


100%|██████████| 8371/8371 [15:43<00:00,  8.87it/s]  


In [96]:
# lists to save artists and artists-uri
artists_found = []
artists_uri = []

In [97]:
# extraction of artists-uri
for i in tqdm(range(len(track_info_list))):
    track_content = track_info_list[i]
    if 'error' not in track_content:
        artist_name = track_info_list[i]['album']['artists'][0]['name']
        artists_found.append(artist_name)
        #print(artist_name)
        artist_uri = track_info_list[i]['album']['artists'][0]['uri']
        artist_uri = artist_uri.split(':')
        artist_uri_id = artist_uri[2]
        artists_uri.append(artist_uri_id)
        #print(artist_uri_id)



100%|██████████| 8371/8371 [00:00<00:00, 373242.18it/s]


In [100]:
df_artist_uri = pd.DataFrame(list(zip(artists_found, artists_uri)), columns =['artist', 'artist_uri'])

In [8]:
df_artist_uri.head()

In [7]:
# list with artists-uri to make calls to artists-endpoint
artist_uri_list = df_artist_uri.artist_uri.tolist()

In [18]:
# list to store information about each artist
artists_info_list = []

In [19]:
# loop for each api-call of the artists-endpoint
for a in tqdm(artist_uri_list):
    artist_info = requests.get(BASE_URL + 'artists/' + a, headers=headers)
    artist_info_json = artist_info.json()
    artists_info_list.append(artist_info_json)

100%|██████████| 7078/7078 [12:31<00:00,  9.42it/s]


In [22]:
# list to store extracted info of artists
artists_name = []
artists_genre = []
artists_popularity = []
artists_id = []
artists_type = []
artists_followers = []



In [23]:
# loop to extract info of each artists
for i in tqdm(range(len(artists_info_list))):
    artist_info = artists_info_list[i]
    artist_name = artist_info['name']
    artists_name.append(artist_name)
    artist_genre = artist_info['genres']
    artists_genre.append(artist_genre)
    artist_popularity = artist_info['popularity']
    artists_popularity.append(artist_popularity)
    artist_type = artist_info['type']
    artists_type.append(artist_type)
    artist_id = artist_info['id']
    artists_id.append(artist_id)
    artist_followers = artist_info['followers']['total']
    artists_followers.append(artist_followers)



100%|██████████| 7078/7078 [00:00<00:00, 425075.65it/s]


In [24]:
# dataframe with info to each artist
df_artist_info = pd.DataFrame(list(zip(artists_name, artists_id, artists_type, artists_popularity, artists_genre, artists_followers)), columns =['artist_name', 'artist_id', 'artist_type', 'artist_popularity', 'artist_genre', 'artist_followers'])

In [25]:
df_artist_info.head()

,artist_name,artist_id,artist_type,artist_popularity,artist_genre,artist_followers
0,Wilder Gray,74RlyWfSIpHWsObpkuc0KC,artist,21,[],388
1,Nick Jonas,4Rxn7Im3LGfyRkY2FlHhWi,artist,64,"[dance pop, edm, pop, post-teen pop]",8570586
2,Eric Chase,6UaMv59B64t5wi1vH4H5HT,artist,37,[pop house],6113
3,Kalash,3J7r4VsNmuWixU0nXvyPd8,artist,64,"[french hip hop, pop urbaine, rap antillais, trap antillais]",504164
4,Various Artists,0LyfQWJT6nXafLPZqxe9Of,artist,0,[],1697769


In [63]:
# merge additional info of each artist to the main streaming history dataframe
df_music_final = df_music.merge(df_artist_info, on='master_metadata_album_artist_name', how='left')

In [9]:
df_music_final.head()

In [35]:
# renaming of files
df_music_final.rename({'ts' : 'play_date', 'master_metadata_album_artist_name' : 'artist_name', 'master_metadata_album_album_name' : 'album_name', 'master_metadata_track_name' : 'track_name', 'uri' : 'track_uri', 'artist_id' : 'artist_uri'}, axis = 1, inplace = True)


In [135]:
# function to find the most precise genre of each artist to select only one genre to each artist
def find_shortest_string(genre_list):
    """
    find the shortest string in list of strings
    :param list genre_list: list of different genres associated with the artist
    :return: shortest and most precise genre
    """
    shortest_strings = []
    if isinstance(genre_list, float):
        return None
    else:
        if len(genre_list) >= 1:
            min_word = min(genre_list, key=len)
            len_min_word = len(min_word)
            for genre in genre_list:
                if len(genre) == len_min_word:
                    shortest_strings.append(genre)
                else:
                    pass
            return random.choice(shortest_strings)



In [142]:
# apply function to select the most precise genre to each artist
df_music_final["artist_genre"]= df_music_final["artist_genre"].fillna("[]")
df_music_final["artist_genre"] = df_music_final["artist_genre"].apply(eval)
df_music_final["single_genre"] = df_music_final["artist_genre"].apply(find_shortest_string)

In [10]:
df_music_final.head(15)

In [12]:
# extract time features of the play_date column
df_music_final['date'] = pd.DatetimeIndex(df_music_final['play_date']).date
df_music_final['time'] = pd.DatetimeIndex(df_music_final['play_date']).time
df_music_final['week'] = pd.DatetimeIndex(df_music_final['play_date']).week
df_music_final['weekday'] = pd.DatetimeIndex(df_music_final['play_date']).weekday
df_music_final['artist_popularity'] = df_music_final['artist_popularity'].fillna(0).astype('int')
df_music_final['artist_followers'] = df_music_final['artist_followers'].fillna(0).astype('int')

In [14]:
# drop not important columns
df_music_final.drop(['username', 'spotify_track_uri', 'ip_addr_decrypted', 'user_agent_decrypted', 'offline_timestamp', 'artist_type'], axis=1, inplace=True)

In [ ]:
# renaming of columns
df_music_final.rename({'ts' : 'play_date', 'master_metadata_album_artist_name' : 'artist_name', 'master_metadata_album_album_name' : 'album_name', 'master_metadata_track_name' : 'track_name', 'uri' : 'track_uri', 'artist_id' : 'artist_uri'}, axis = 1, inplace = True)

In [18]:
# rearranging
df_music_final = df_music_final[['play_date', 'year', 'date', 'time', 'week', 'weekday', 'platform', 'artist_name', 'album_name', 'track_name', 'track_uri', 'ms_played', 'seconds_played', 'minutes_played', 'conn_country', 'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline', 'incognito_mode', 'artist_uri', 'artist_popularity', 'artist_genre','single_genre', 'artist_followers']]

In [24]:
df_music_final.head()

,play_date,year,date,time,week,weekday,platform,artist_name,album_name,track_name,track_uri,ms_played,seconds_played,minutes_played,conn_country,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,artist_uri,artist_popularity,artist_genre,single_genre,artist_followers
0,2019-08-02 05:16:32+00:00,2019,2019-08-02,05:16:32,31,4,"iOS 12.3.2 (iPhone10,2)",Major Lazer,Can't Take It From Me,Can't Take It From Me - Paul Woolford Remix,2A8uxtb4LEXvwukVVMeBU4,6293,6.293,0.104883,DE,clickrow,endplay,False,NaN,False,False,738wLrAtLtCtFOLvQBXOXp,74,"['dance pop', 'edm', 'electro house', 'moombahton', 'pop', 'pop dance', 'pop rap', 'tropical house']",edm,6320502
1,2021-07-09 19:26:10+00:00,2021,2021-07-09,19:26:10,27,4,"iOS 14.6 (iPhone12,1)",GT_Ofice,Ooh La La,Ooh La La,3lQiZXNB9YoxgCBtXjZjGm,26871,26.871,0.447850,HR,playbtn,logout,True,NaN,False,False,0KdM0f9BzAkPzHFxxnn2tS,36,[],NaN,2803
2,2013-06-09 12:40:58+00:00,2013,2013-06-09,12:40:58,23,6,Windows 7 (Unknown Ed) SP1 [x86 0],Michael Mind Project,Kontor Presents Michael Mind Project - State of Mind,Love's Gonna Get You - G&G Remix Edit,5zTF59Ft413797mxMQF1PY,167990,167.990,2.799833,DE,popup,trackdone,False,0.0,False,False,3I4ttwmYBwsR01fin6ItJR,42,"['hands up', 'pop house']",hands up,44026
3,2020-11-16 19:50:06+00:00,2020,2020-11-16,19:50:06,47,0,Partner amazon_salmon Amazon;Echo_Dot;27d4dfe427b34d57995b463e5d63198d;;tpapi,Nordfold,Down Memory Lane EP,Rituals,0bzNhNVnXxKtrPhH2ragS9,200231,200.231,3.337183,AT,trackdone,trackdone,False,NaN,False,False,2QAapUP6tz100gnoEb0ITc,29,[],NaN,2699
4,2022-06-23 19:14:10+00:00,2022,2022-06-23,19:14:10,25,3,"iOS 15.5 (iPhone12,1)",Alan Walker,Avem (The Aviation Theme),Avem (The Aviation Theme),0NBD4g9Zwy1rMAMWBYS6Ju,38440,38.440,0.640667,DE,fwdbtn,fwdbtn,True,NaN,False,False,7vk5e3vY1uw9plTHJAMwjN,81,['electro house'],electro house,36329657


In [25]:
# save cleaned dataframe to csv-file
df_music_final.to_csv('music_streaming_data_clean.csv', encoding="utf-8", index=False)